- 기본 시퀀스 프로토콜 : __len__()과 __get_item__() 메서드
- 여러 항목을 가진 객체를 안전하게 표현
- 슬라이싱을 지원해서 새로운 백터 객체 생성
- 포함된 요소 값을 모두 고려한 집합 예상
- 커스터마이즈된 포맷 언어 확장

### 10.1 Vector: 사용자 정의 시퀀스형

### 10.2 Vector 버전 #1: Vector2d 호환

In [37]:
from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
    
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components =  reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x*x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indeces must be integers'
            raise TypeError(msg.format(cls=cls))
        return self._components[index]
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:], cast(typecode))
        return cls(memv)

### 10.3 프로토콜과 덕 타이핑

In [38]:
import collections
Card = collections.namedtuple('Card',['rank','suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self):
        return self._cards[position]

### 10.4 Vector 버전 #2: 슬라이스 가능한 시퀀스

In [39]:
v1 = Vector([3,4,5])
print (len(v1))
print (v1[0],v1[-1])
v7 = Vector(range(7))
print (v7[1:4])

3
3.0 5.0
(1.0, 2.0, 3.0)


#### 10.4.1 슬라이싱의 작동 방식

In [21]:
class MySeq:
    def __getitem__(self, index):
        return index

In [27]:
s = MySeq()
print (s[1])
print (s[1:4])
print (s[1:4:2])
print (s[1:4:2, 9])
print (s[1:4:2, 7:9])

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


In [33]:
print (slice)
print (dir(slice))
print (help(slice.indices))

<class 'slice'>
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']
Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.

None


In [40]:
print (slice(None, 10 ,2 ).indices(5))
print (slice(-3, None, None).indices(5))

(0, 5, 2)
(2, 5, 1)


#### 10.4.2 슬라이스를 인식하는 \__getitem\__()

In [44]:
v7 = Vector(range(7))
print (v7[-1])
print (v7[1:4])
print (v7[1:4])
print (v7[-1:])
print (v7[1,2])

6.0
(1.0, 2.0, 3.0)
(1.0, 2.0, 3.0)
(6.0,)


TypeError: Vector indeces must be integers